In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [ ]:
!pip install nilearn

In [ ]:
# 健常者グループの被験者IDのリスト
healthy_subject_ids = ['4101', '4103', '4106', '4107', '4109', '4110', '4111', '4112', '4113', '4114', '4115', '4116', '4117', '4118', '4119', '4120', '4121', '4122', '4123', '4125', '4126', '4128', '4129', '4130', '4131']

# パーキンソン病患者グループの被験者IDのリスト
patient_subject_ids = ['4201', '4204', '4205', '4206', '4207', '4208', '4210', '4211', '4212', '4213', '4214', '4215', '4217', '4218', '4219', '4220', '4221', '4224', '4225', '4226', '4227']

In [ ]:
# sessionのリスト
ses_ids = ['1', '2']

In [ ]:
# runsのリスト
run_ids = ['1', '2', '3', '4', '5', '6']

In [ ]:
vs_titles = ["alerting", "executive", "orienting"]

In [ ]:
import os

directory = "/content/drive/MyDrive/ANT"
if not os.path.exists(directory):
  os.makedirs(directory)

# ディレクトリが存在しない場合にのみ作成
for vs_title in vs_titles:
  vs_directory = directory + "/" + vs_title
  if not os.path.exists(vs_directory):
      os.makedirs(vs_directory)

  for subject_id in healthy_subject_ids+patient_subject_ids:
    sub_directory = vs_directory + "/sub-RC" + subject_id
    if not os.path.exists(sub_directory):
      os.makedirs(sub_directory)

    for ses_id in ses_ids:
      ses_directory = sub_directory + "/ses-" + ses_id
      if not os.path.exists(ses_directory):
        os.makedirs(ses_directory)

In [ ]:
import os
for subject_id in healthy_subject_ids + patient_subject_ids:
  for ses_id in ses_ids:
    for run_id in run_ids:
      func_file = f'https://s3.amazonaws.com/openneuro.org/ds001907/derivatives/sub-RC{subject_id}/ses-{ses_id}/func/sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_bold.nii.gz'
      os.system(f"wget {func_file}")
      event_file = f'https://s3.amazonaws.com/openneuro.org/ds001907/sub-RC{subject_id}/ses-{ses_id}/func/sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_events.tsv'
      os.system(f"wget {event_file}")


In [ ]:
from nilearn.image import get_data
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_stat_map

def unit_glm(subject_id, ses_id, run_id):
  t_r = 2.4
  sigma = 16.5
  high_pass = 1 / (2 * t_r * sigma)

  glm = FirstLevelModel(t_r=t_r, high_pass=high_pass, smoothing_fwhm=4)

  # ファイル名、イベントファイル名を取得
  func_filename = f'sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_bold.nii.gz'
  event_filename = f'sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_events.tsv'

  # fMRIデータの読み込み
  fmri_img = nib.load(func_filename)

  # デザインマトリックスの読み込み
  events = pd.read_csv(event_filename, delimiter='\t')

  # GLMをデータに適用
  glm = glm.fit(fmri_img, events=events)

  contrasts = {
      'alerting': 'CenterCue - NoCue',
      'orienting': 'SpatialCue - CenterCue',
      'executive': 'Incongruent - Congruent'
  }

  # google driveに結果を保存する
  for name, expression in contrasts.items():
      contrast = glm.compute_contrast(expression)
      output_filename = f"/content/drive/MyDrive/ANT/{name}/sub-RC{subject_id}/ses-{ses_id}/{func_filename}"
      nib.save(contrast, output_filename)

In [ ]:
# 各被験者についてGLM分析を実行し、結果をGoogle Driveに追加
for subject_id in healthy_subject_ids + patient_subject_ids:
  for ses_id in ses_ids:
    for run_id in run_ids:
      unit_glm(subject_id, ses_id, run_id)